In [1]:
# Time plots
from jupyterplot import ProgressPlot
import numpy as np
import time

# Testing
import pytest
import ipytest
ipytest.autoconfig()

In [2]:
def get_time(func, args):
    start = time.perf_counter()
    result = func(*args)
    end = time.perf_counter()
    print(f"The result of calling {func.__name__} with {args} is {result}")
    return end - start


def animate_times(funcs, test_cases):
    pp = ProgressPlot(line_names=[func.__name__ for func in funcs],
                  x_lim=[0, len(test_cases)],
                    y_lim = [0, 20]
                  )
    for test in test_cases:
        pp.update([[get_time(func, args=test)for func in funcs]])
    pp.finalize()


def animate(funcs, test_cases):
    pp = ProgressPlot(line_names=[func.__name__ for func in funcs],
                  x_lim=[0, len(test_cases)],
                    y_lim = [0, 100]
                  )
    
    for test in test_cases:
        pp.update([[func(test)for func in funcs]])
    pp.finalize()

def linear(n): return n
def log(n) : return np.log(n)
def square(n) : return n ** 2
def cube(n) : return n ** 3
def exponential(n) : return np.exp(n)
def fact(n):
    if n <= 1: return n
    return n * fact(n-1)


# Dummy Run
funcs = [log, linear, square, cube, exponential]
test_cases = np.linspace(0.000001,100, num=1000, endpoint=False)
animate(funcs, test_cases)

### Q1. Grid Traveler

In [3]:
def grid_traveler(m, n):
    if m == 0 or n == 0: return 0
    if m == 1 or n== 1: return 1
    return grid_traveler(m-1, n) + grid_traveler(m, n-1)

def grid_traveler_dp(m, n, memo={}):
    key = m, n
    if key in memo: return memo[key]
    if m == 0 or n == 0: return 0
    if m == 1 or n== 1: return 1
    
    memo[key] = grid_traveler_dp(m-1, n, memo) + grid_traveler_dp(m, n-1, memo)
    return memo[key]

In [4]:
%%run_pytest -qq

@pytest.mark.parametrize("m,n,expected", [
    (1, 1, 1),
    (2, 3, 3),
    (3, 2, 3),
    (3, 3, 6),
#     (18, 18, 2333606220), # this will take too much time that i don't got
])
def test_grid_traveler(m, n, expected):
    assert grid_traveler(m, n) == expected

@pytest.mark.parametrize("m,n,expected", [
    (1, 1, 1),
    (2, 3, 3),
    (3, 2, 3),
    (3, 3, 6),
    (18, 18, 2333606220),
])    
def test_grid_traveler_memoized(m, n, expected):    
    assert grid_traveler_dp(m, n, memo={}) == expected

.........                                                                                                                                                                                                                        [100%]


In [5]:
animate_times(funcs= [grid_traveler, grid_traveler_dp] , test_cases = [(i, i) for i in range(15)])

The result of calling grid_traveler with (0, 0) is 0
The result of calling grid_traveler_dp with (0, 0) is 0
The result of calling grid_traveler with (1, 1) is 1
The result of calling grid_traveler_dp with (1, 1) is 1
The result of calling grid_traveler with (2, 2) is 2
The result of calling grid_traveler_dp with (2, 2) is 2
The result of calling grid_traveler with (3, 3) is 6
The result of calling grid_traveler_dp with (3, 3) is 6
The result of calling grid_traveler with (4, 4) is 20
The result of calling grid_traveler_dp with (4, 4) is 20
The result of calling grid_traveler with (5, 5) is 70
The result of calling grid_traveler_dp with (5, 5) is 70
The result of calling grid_traveler with (6, 6) is 252
The result of calling grid_traveler_dp with (6, 6) is 252
The result of calling grid_traveler with (7, 7) is 924
The result of calling grid_traveler_dp with (7, 7) is 924
The result of calling grid_traveler with (8, 8) is 3432
The result of calling grid_traveler_dp with (8, 8) is 3432
T

#### Complexity Analysis

### Q2 Can Sum 

In [6]:
def can_sum(target_sum, numbers):
    if target_sum == 0: return True
    if target_sum < 0: return False
    
    for number in numbers:
        remainder = target_sum - number
        subanswer = can_sum(remainder, numbers)
        
        if subanswer:
            return subanswer
    
    return False

def can_sum_memo(target_sum, numbers, memo={}):
    key = target_sum
    
    if key in memo: return memo[key]
    if target_sum == 0: return True
    if target_sum < 0: return False
    
    for number in numbers:
        remainder = target_sum - number
        
        if can_sum_memo(remainder, numbers, memo): 
            memo[target_sum] = True
            return True
    
    memo[target_sum]=False
    return False

In [7]:
%%run_pytest -qq

@pytest.mark.parametrize("target_sum,numbers", [
    (7, [5, 3, 4, 2]),
    (8, [2, 3, 5])
])
def test_can_sum_true(target_sum, numbers):
    assert can_sum_memo(target_sum, numbers, memo={})

    
@pytest.mark.parametrize("target_sum,numbers", [
    (7, [2, 4]),
    (8, [7, 14])
])
def test_cannot_sum(target_sum, numbers):
    assert not can_sum_memo(target_sum, numbers, memo={})

.............                                                                                                                                                                                                                    [100%]


#### Complexity Analysis

### Q3 How Sum

In [8]:
def how_sum(target_sum, numbers):
    if target_sum == 0: return ""
    if target_sum < 0: return None
    
    for number in numbers:
        remainder = target_sum - number
        how_sum_remainder = how_sum(remainder, numbers)
        
        if how_sum_remainder is not None:
            if how_sum_remainder == "": return str(number)
            return f"{number}, {how_sum_remainder}"
    return None


def how_sum_memo(target_sum, numbers, memo={}):
    key = target_sum
    
    if key in memo: return memo[key]
    if target_sum == 0: return ""
    if target_sum < 0: return None
    
    for number in numbers:
        remainder = target_sum - number
        how_sum_remainder = how_sum_memo(remainder, numbers, memo)
        if how_sum_remainder is not None:
            memo[key] = f"{number}, {how_sum_remainder}" if how_sum_remainder != "" else str(number)
            return memo[key]
    
    memo[key] = None
    return None

In [9]:
%%run_pytest -qq

@pytest.mark.parametrize("target_sum,numbers,expected", [
    (7, [2, 3], "2, 2, 3"),
    (7, [5, 3, 4, 7], "3, 4"),
    (7, [2, 4], None),
    (8, [2, 3, 5], "2, 2, 2, 2"),
#     (300, [7, 14], "None"), # takes alot of time
])

def test_how_sum(target_sum, numbers, expected):    
    assert how_sum(target_sum, numbers) == expected
    
@pytest.mark.parametrize("target_sum,numbers,expected", [
    (7, [2, 3], "2, 2, 3"),
    (7, [5, 3, 4, 7], "3, 4"),
    (7, [2, 4], None),
    (8, [2, 3, 5], "2, 2, 2, 2"),
    (300, [7, 14], None),
])
def test_how_sum_memo(target_sum, numbers, expected):    
    assert how_sum_memo(target_sum, numbers, memo={}) == expected

......................                                                                                                                                                                                                           [100%]


#### Complexity Analysis

### Q4 Best Sum

In [10]:
def best_sum(target_sum, numbers):
    if target_sum == 0: return []
    if target_sum < 0: return None
    
    best_path = None
    for number in numbers:
        remainder = target_sum - number
        best_sum_remainder = best_sum(remainder, numbers)
        
        if best_sum_remainder is not None:
            current_path = best_sum_remainder + [number]
            if best_path is None or len(best_path) > len(current_path):
                best_path = current_path
    
    return best_path


def best_sum_memo(target_sum, numbers, memo={}):
    key = target_sum
    if key in memo: return memo[key]
    if target_sum == 0: return []
    if target_sum < 0: return None
    
    
    best_path = None
    
    for number in numbers:
        remainder = target_sum - number
        best_sum_remainder = best_sum_memo(remainder, numbers, memo)
        
        if best_sum_remainder is not None:
            current_path = best_sum_remainder + [number]
            if best_path is None or len(current_path) < len(best_path):
                best_path = current_path
                memo[key] = best_path
    
    memo[key] = best_path
    return memo[key]        

In [11]:
%%run_pytest -qq

@pytest.mark.parametrize("target_sum,numbers,expected", [
    (7, [2, 3], [3, 2, 2]),
    (7, [5, 3, 4, 7], [7]),
    (7, [2, 4], None),
    (8, [2, 3, 5], [5, 3]),
#     (300, [7, 14], None),
])
def test_best_sum(target_sum, numbers, expected):    
    assert best_sum(target_sum, numbers) == expected
    
@pytest.mark.parametrize("target_sum,numbers,expected", [
    (7, [2, 3], [3, 2, 2]),
    (7, [5, 3, 4, 7], [7]),
    (7, [2, 4], None),
    (8, [2, 3, 5], [5, 3]),
    (300, [7, 14], None),
])
def test_best_sum_memo(target_sum, numbers, expected):    
    assert best_sum_memo(target_sum, numbers, memo={}) == expected

...............................                                                                                                                                                                                                  [100%]


#### Complexity Analysis

### Q5 Can Construct

In [12]:
def can_construct(target, choices):
    if target == "": return True
    
    for choice in choices:
        if target.startswith(choice):
            sub = can_construct(target[len(choice):], choices)
            if sub:
                return True
    return False


def can_construct_memo(target, choices, memo={}):
    key = target
    if key in memo: return memo[key]
    if target == "": return True
    
    for choice in choices:
        if target.startswith(choice):
            sub = can_construct_memo(target[len(choice):], choices, memo)
            if sub:
                memo[key]=True
                return True
    memo[key] = False
    return False
    

In [13]:
%%run_pytest -qq

@pytest.mark.parametrize("target_string,choices", [
    ('abcdef', ['ab', 'abc', 'cd', 'ef', 'de']),
    ('enterapotentpot', ['a', 'p', 'enter', 'ent', 'ot', 'o', 'p'])
])
def test_can_construct_true(target_string, choices):
    assert can_construct_memo(target_string, choices, memo={})


@pytest.mark.parametrize("target_string,choices", [
    ('abcdef', ['ab', 'abc', 'cd', 'de']),
])
def test_cannot_construct(target_string, choices):
    assert not can_construct_memo(target_string, choices, memo={})

..................................                                                                                                                                                                                               [100%]


#### Complexity Analysis